# Работа с API

In [1]:
import time
from datetime import datetime
import requests
import pandas as pd

In [2]:
# функция для получения количества друзей пользователя
def friends_count(id):
    try:
        url = 'https://api.vk.com/method/friends.get' 
        params = {'user_id': id, 'v': 5.99, 'access_token': token, 'lang': 'ru'}
        count = requests.get(url, params=params)
        time.sleep(0.2)
        return count.json()['response']['count']
    except KeyError:
        return 0

In [3]:
# преобразование временной метки в объект datetime
def last_seen_time(last_seen):  
    day_time = datetime.fromtimestamp(last_seen)
    return day_time

In [4]:
# высчитывание возраста пользователя
def b_age(bdate):
    try:
        bdate = str(bdate)
        if len(bdate) > 5:
            # формат даты
            date_format = '%d.%m.%Y'
            # форматирование даты рождение
            birth_date = datetime.strptime(bdate, date_format)
            # получение текущего времени
            current_date = datetime.now()
        
            # вычисление разницы в годах с учетом того, наступил день рождения уже или нет
            age = current_date.year - birth_date.year - \
                ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
            return age
        else:
            return 0
    except ValueError:
        return None  

In [5]:
def dataframe_create(members_raw, city_row):
    # перенос данных о пользователях в Pandas DataFrame
    members = pd.json_normalize(members_raw)
    members = members[['id', 'fullname', 'first_name','last_seen', 'mobile_phone', 'friends_count', 'town', 'bdate']]

    # перенос данных о городах в Pandas DataFrame
    merged_dict = {}
    for row in city_row:
        merged_dict.update(row)

    # Создание DataFrame из объединенного словаря
    city = pd.DataFrame(merged_dict.items(), columns=['city_id', 'name'])

    return members, city

In [6]:
def dataframe_transform(members):
    # переименование столбцов в соответствии с заданием
    members = members.rename(columns={'id': 'user_id_vk'})
    members = members.rename(columns={'mobile_phone': 'contacts'})
    members = members.rename(columns={'bdate': 'age'})
    members = members.rename(columns={'first_name': 'name'})

    # замена одного апострофа двумя, для дальнейшей загрузки в БД
    members['fullname'] = members['fullname'].str.replace("'", "''")
    members['name'] = members['name'].str.replace("'", "''")

    # применение функции для высчитывания возраста пользователей
    members['age'] = members['age'].apply(b_age)

    # заполнение пустот 0
    members.fillna(0, inplace=True);

    # изменение типа данных
    members['age'] = members['age'].astype(int)
    members['friends_count'] = members['friends_count'].astype(int)

    # форматирование в дату
    members['last_seen'] = pd.to_datetime(members['last_seen'])

    return members

In [7]:
# сохранение данных (порционно) 
def dataframe_save(members, city):
    try:
        # чтение данных
        exist_members = pd.read_csv('VK_Fishing_members')
        exist_city = pd.read_csv('VK_Fishing_city')
    except FileNotFoundError:
        # создание файлов с новыми данными
        members.to_csv('../CSV/VK_Fishing_members', index=False)
        city.to_csv('../CSV/VK_Fishing_city', index=False)
    else:
        # конкатенация данных
        combi_members = pd.concat([exist_members, members], ignore_index=True)
        combi_city = pd.concat([exist_city, city], ignore_index=True)
        # сохранение данных
        combi_members.to_csv('../CSV/VK_Fishing_members', index=False)
        combi_city.drop_duplicates(inplace=True)
        combi_city.to_csv('../CSV/VK_Fishing_city', index=False)

In [8]:
# получение данных

token = ''
url = 'https://api.vk.com/method/groups.getMembers'
count = 1000
offset = 0 
members_raw = [] 
max_count = 100000 
counter = 1
while offset < max_count: 
    print(f'Выгрузка {count * counter} пользователей с offset = {offset}')   
    params = {'group_id': 'vk_fishing', 'v': 5.99, 'fields': 'bdate, last_seen, city, contacts', 'access_token': token, 'offset': offset, 'lang': 'ru'}
    response = requests.get(url, params=params)
    data = response.json() ['response']['items']
    members_raw += data
    # увеличивение смещения на количество строк, которое уже выгружено 
    offset += count 
    counter += 1

Выгрузка 1000 пользователей с offset = 0
Выгрузка 2000 пользователей с offset = 1000
Выгрузка 3000 пользователей с offset = 2000
Выгрузка 4000 пользователей с offset = 3000
Выгрузка 5000 пользователей с offset = 4000
Выгрузка 6000 пользователей с offset = 5000
Выгрузка 7000 пользователей с offset = 6000
Выгрузка 8000 пользователей с offset = 7000
Выгрузка 9000 пользователей с offset = 8000
Выгрузка 10000 пользователей с offset = 9000
Выгрузка 11000 пользователей с offset = 10000
Выгрузка 12000 пользователей с offset = 11000
Выгрузка 13000 пользователей с offset = 12000
Выгрузка 14000 пользователей с offset = 13000
Выгрузка 15000 пользователей с offset = 14000
Выгрузка 16000 пользователей с offset = 15000
Выгрузка 17000 пользователей с offset = 16000
Выгрузка 18000 пользователей с offset = 17000
Выгрузка 19000 пользователей с offset = 18000
Выгрузка 20000 пользователей с offset = 19000
Выгрузка 21000 пользователей с offset = 20000
Выгрузка 22000 пользователей с offset = 21000
Выгрузка 2

In [ ]:
count = 0 

# формирование данных для таблицы с городами
city_row = [{-1: 'Отсутствует'}]

# обработка выгруженных данных
for member in members_raw:
    # добавление в основную таблицу индекс города,
    # а в таблицу city индекс и город
    if 'city' in member:
        member['town'] = member['city']['id']
        city_id = {member['city']['id']: member['city']['title']}
        if city_id not in city_row:
            city_row.append(city_id)
    else:
        member['town'] = -1
    # преобразование временной метки 
    if 'last_seen' in member:
        member['last_seen'] = last_seen_time(member['last_seen']['time'])

    # создание нового столбца состоящий из имени и фамилии
    member['fullname'] = f'{member["first_name"]} {member["last_name"]}'

    # получение данных о количестве друзей пользователя
    if member['can_access_closed'] == True:
        member['friends_count'] = friends_count(member['id'])
    else:
        member['friends_count'] = None

    # обработка и сохранение данных (порционно)
    if count % 1000 == 0 and count > 0:
        start = count - 1000
        members_temporarily, city_temporarily = dataframe_create(members_raw[start:count], city_row)
        members_temporarily = dataframe_transform(members_temporarily)
        dataframe_save(members_temporarily, city_temporarily)
    
    count += 1